In [1]:
from db import call_db, call_ignition, call_db_json, update_ignition, update_db, force_cancel_rail
from ml_model import upcomming_demand_prediction, dev, model, cols, flag_list, family_mapper, model_mapper, cols_b
from queries import calc_all_priority, all_priority_call, get_kanban, set_kanban, stock_all
from util_func import ProcessingStorage, DemandStorage, StockStorage, DiecastStorage
import pandas as pd
import numpy as np
import difflib
import re
import math
import psycopg2
from configparser import ConfigParser
from psycopg2.extras import RealDictCursor
# import torch, difflib, json, os, math
from datetime import timedelta, date, datetime

throttle_2880_2841.pth


In [16]:
test = {'release_id': 1,
    'item_description': "sdg",
    'memo': "sdg",
    'writer': "sdg",
    'former_release_status': True,
    'udpated_release_status': True,
    'former_release_quantity': 1,
    'updated_release_quantity': 1}
test['date'] = datetime.now()
len_test = len(test.values())
string = str(['%s'] * len_test)[1:-1].replace("'", "")
string

'%s, %s, %s, %s, %s, %s, %s, %s, %s'

In [ ]:
(str(list(test.keys())).replace('\'', '')[1:-1])
list(test.values())


In [99]:
set_kanban()
stock = StockStorage()
sql = f"""
        SELECT product_family, product_model,item_number, item_description, product_length, SUM(quantity) AS QTY, virtual_location FROM current_wip_inventory
            JOIN wip_inventory_states on state_id = id
            WHERE current_wip_inventory.state_id = (SELECT id from wip_inventory_states order by timestamp desc limit 1)
                AND virtual_location in ('GY1', 'GY3', 'GY4')
                AND rail = true
                AND warehouse_location != 'INTRAN'
                AND current_wip_inventory.product_length != 'NaN'
                AND item_description not like '%MML%'
                AND quantity != 0
            GROUP BY (product_family, product_model, item_number, item_description, product_length, virtual_location) Order By product_family, product_model, QTY DESC;
        """
stock.set_data(call_db_json(sql))

gy4_stock = stock.GY4.query("family in ('HRW', 'SHW') and model in ('27', '35') and length == 3000")
SHW_HRW[SHW_HRW['item_description'].str.contains()]

,family,model,item_number,item_description,length,qty,gy,total_length
20,HRW,27,806227200C30000,HRW27-3000L (GK)GROUND RAIL,3000.0,45.0,GY4,135.0
25,HRW,35,806235200C30000,HRW35-3000L (GK)GROUND RAIL,3000.0,130.0,GY4,390.0
235,SHW,27,806827200C30000,SHW27-3000L (GK)GROUND RAIL,3000.0,196.0,GY4,588.0
245,SHW,35,806835200C30000,SHW35-3000L (GK)GROUND RAIL,3000.0,21.0,GY4,63.0


In [205]:

def release_check_sum(df, gy4_stock):
    pack_size_df = pd.read_csv('./etc/pack_qty.csv')
    short_standard = ['HSR35-1320', 'HSR35-2040', 'HSR45-1200', 'HSR45-1778']        
    SHW_HRW = stock.GY4.query("family in ('HRW', 'SHW') and model in ('27', '35') and length == 3000")
    df['force_release'] = False    
    df['from_gy4'] = False    
    for r in df.itertuples():
        rail_type, to_be_parsed = r.des.split('-')
        rail_length = int(to_be_parsed.split('L')[0])
        width = int(rail_type[-2:])        
        type_length = rail_type + '-' + str(rail_length)
        temp_stock = r.stock        
        ## get the stock of SHW,HRW rails for counting stock from GY4 not GY3
        ## to manipulate tuple, had to allocate a temp_release_qty variable.
        if rail_length < 3000:
            if type_length in short_standard:
                flag = 'STANDARD'
                temp_release_qty = pack_size_df[(pack_size_df['type'] == rail_type) & (
                    pack_size_df['length'] == rail_length)]['qty'].values[0]
            else:
                if rail_type == 'HSR45':
                    if rail_length < 3000:
                        temp_release_qty = r[5]
                flag = 'SHORT'
        elif rail_length > 3240:
            temp_release_qty = r[5]
            flag = 'LONG'
        else:
            try:
                temp_release_qty = pack_size_df[(pack_size_df['type'] == rail_type) & (
                    pack_size_df['length'] == 3000)]['qty'].values[0]
                flag = 'STANDARD'
            except:                
                continue
                temp_stock = int(SHW_HRW[SHW_HRW['item_description'].str.contains(rail_type)]['qty'].values[0])                
                df.loc[r[0], 'stock'] = temp_stock        
                df.loc[r[0], 'from_gy4'] = True
                flag = '3000_GY4'
                temp_release_qty = r[5]
                
        DEMAND = r[4] if flag == 'STANDARD' else r[5]        
        release_qty = abs(
            temp_release_qty * math.floor(DEMAND / temp_release_qty)) if flag == 'STANDARD' else abs(temp_release_qty)        
        pack_size = temp_release_qty if flag == 'STANDARD' else 0
        if flag == 'STANDARD' and width <= 25 and 0 < r[4] < 100 and r[5] < -100:
            necessary = r[4] - 200            
            release_qty = abs(temp_release_qty * math.floor(necessary / temp_release_qty))            
            df.loc[r[0], 'force_release'] = True
        in_stock = 'LOW'
        if DEMAND + temp_stock > 0:
            in_stock = 'FULL'        
        elif ((DEMAND * 0.7) + temp_stock > 0) & (temp_stock >= pack_size):
            in_stock = 'PARTIAL'
            release_qty = temp_stock            
        df.loc[r[0], 'type'] = rail_type
        df.loc[r[0], 'length'] = rail_length
        df.loc[r[0], 'pack_size'] = pack_size
        df.loc[r[0], 'flag'] = flag
        df.loc[r[0], 'in_stock'] = in_stock
        df.loc[r[0], 'release_qty'] = release_qty
        df.loc[r[0], 'release_meter'] = int(release_qty * rail_length / 1000)
    return df['release_meter'].sum(), df

In [208]:
def get_gy1_release():
    kanban = get_kanban().set_index('des')
    w8 = kanban.columns[12]
    w14 = kanban.columns[18]
    df = kanban[kanban[w14] < 0].sort_values(
        w8).iloc[:, [1, 2, 12, 18]].reset_index()
    total_demand, df = release_check_sum(df, stock.GY4)
    df = df.dropna(subset=['flag'])

    df8 = df[df[w8] < 0]
    df14 = df[(df[w8] >= 0) & df['in_stock']]
    df14 = df14.sort_values('release_meter', ascending=False)
    df = pd.concat([df8, df14])
    df = df.rename(columns={w8: 'w8', w14: 'w14'})
    production_amount = 0
    df['release'] = False
    for r in df.itertuples():        
        if (r.in_stock != 'LOW' and (r.w8 < 0 or r.force_release)):
            production_amount += r.release_meter            
            df.loc[r[0], 'release'] = True        
        # if production_amount > total_demand * 0.5:
        #     break    
    df['date'] = datetime.now()
    df = df.rename(columns={'des':'item_description'})
    df['memo'] = ''
    df = df.fillna(0)    
    return df
df = get_gy1_release()
df

,item_description,stock,gy3,w8,w14,force_release,from_gy4,type,length,pack_size,flag,in_stock,release_qty,release_meter,release,date,memo
0,SHS35-3020L HALF RAIL 3,294,3,-302,-664,False,False,SHS35,3020.0,36.0,STANDARD,PARTIAL,294.0,887.0,True,2023-03-01 12:38:29.790153,
2,SR25-3000LY HALF RAIL 3,672,525,-105,-332,False,False,SR25,3000.0,96.0,STANDARD,FULL,192.0,576.0,True,2023-03-01 12:38:29.790153,
3,HSR30-3020L HALF RAIL 3,72,23,-52,-273,False,False,HSR30,3020.0,56.0,STANDARD,FULL,56.0,169.0,True,2023-03-01 12:38:29.790153,
4,SR20-3000L HALF RAIL 3 BLANK,0,381,-24,-36,False,False,SR20,3000.0,150.0,STANDARD,LOW,150.0,450.0,False,2023-03-01 12:38:29.790153,
14,SHS20-3001L HALF RAIL 3,900,23,6,-286,True,False,SHS20,3001.0,150.0,STANDARD,FULL,300.0,900.0,True,2023-03-01 12:38:29.790153,
26,SHS30-3020L HALF RAIL 3,44,26,254,-73,False,False,SHS30,3020.0,72.0,STANDARD,FULL,216.0,652.0,False,2023-03-01 12:38:29.790153,
22,HSR45-3020L HALF RAIL 3,602,172,166,-9,False,False,HSR45,3020.0,30.0,STANDARD,FULL,150.0,453.0,False,2023-03-01 12:38:29.790153,
23,HSR35-3020L HALF RAIL 3,392,238,166,-99,False,False,HSR35,3020.0,49.0,STANDARD,FULL,147.0,443.0,False,2023-03-01 12:38:29.790153,
25,HSR45-1200L HALF RAIL 3,521,4,229,-111,False,False,HSR45,1200.0,30.0,STANDARD,FULL,210.0,252.0,False,2023-03-01 12:38:29.790153,
7,HSR25-4100L HALF RAIL 3,0,0,0,-50,False,False,HSR25,4100.0,0.0,LONG,LOW,50.0,205.0,False,2023-03-01 12:38:29.790153,


In [90]:
stock = StockStorage()
sql = f"""
    SELECT product_family, product_model,item_number, item_description, product_length, SUM(quantity) AS QTY, virtual_location FROM current_wip_inventory
        JOIN wip_inventory_states on state_id = id
        WHERE current_wip_inventory.state_id = (SELECT id from wip_inventory_states order by timestamp desc limit 1)
            AND virtual_location in ('GY1', 'GY3', 'GY4')
            AND rail = true
            AND warehouse_location != 'INTRAN'
            AND current_wip_inventory.product_length != 'NaN'
            AND item_description not like '%MML%'
            AND quantity != 0
        GROUP BY (product_family, product_model, item_number, item_description, product_length, virtual_location) Order By product_family, product_model, QTY DESC;
    """
stock.set_data(call_db_json(sql))

SHW_HRW = stock.GY4.query("family in ('HRW', 'SHW') and model in ('27', '35') and length == 3000")
SHW_HRW

In [98]:

# SHW_HRW = gy4.query("family in ('HRW', 'SHW')").groupby(['family', 'model'])



,family,model,item_number,item_description,length,qty,gy,total_length
20,HRW,27,806227200C30000,HRW27-3000L (GK)GROUND RAIL,3000.0,45.0,GY4,135.0
25,HRW,35,806235200C30000,HRW35-3000L (GK)GROUND RAIL,3000.0,130.0,GY4,390.0
235,SHW,27,806827200C30000,SHW27-3000L (GK)GROUND RAIL,3000.0,196.0,GY4,588.0
245,SHW,35,806835200C30000,SHW35-3000L (GK)GROUND RAIL,3000.0,21.0,GY4,63.0


In [81]:
df2 = get_gy1_release()
def config(db, section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    if db == "postgres":
        parser.read("./etc/postgres.ini")
    else:
        parser.read("./etc/ignition.ini")
    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(
            section, db))
    return db


def execute_many_with_df(db, df, table):    
    # Create a list of tupples from the dataframe values
    params = config(db=db)
    conn = psycopg2.connect(**params)
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    num_cols = len(df.columns)
    # SQL query to execute
    # query  = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s)" % (table, cols)
    string = ('%%s,'*num_cols)[:-1]
    query  = f"INSERT INTO %s(%s) VALUES({string})" % (table, cols)
    cursor = conn.cursor()
    try:
        cursor.executemany(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_many() done")
    cursor.close()
table = 'ignition.initial_release.gy_rail_initial_release'
execute_many_with_df('ignition', df2, table)

execute_many() done


In [65]:
def delete_table(db, table):
    conn = None
    sql = f'''
        TRUNCATE {table};
        DELETE FROM {table}    
    '''
    try:
        params = config(db=db)
        conn = psycopg2.connect(**params)
        cur = conn.cursor(cursor_factory=RealDictCursor)
        cur.execute(sql)
        conn.commit()
        return f'{table} was DELETED'
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
delete_table('ignition', 'ignition.initial_release.gy_rail_initial_release')            

'ignition.production_schedule.gy_rail_initial_release was DELETED'

In [66]:
def update_release(item_description, release, memo):    
    sql = f""" UPDATE ignition.initial_release.gy_rail_initial_release
                SET release = '{release}', memo = '{memo}'
                WHERE item_description = '{item_description}'"""
    return update_ignition(sql)
update_release('HSR45-1568L HALF RAIL 3', False, "from python")

'success'

In [ ]:
# import re
# m = re.search('\d{4}', '-3020L')
# w14_df['length'] = w14_df['des'].apply(lambda x: int(re.search('\d{4}', x)[0]))

In [30]:
pack_size

,product,length,qty
0,HSR15,3240,200
1,SHS15,3040,240
2,SR15,3240,240
3,HSR20,3020,130
4,SHS20,3020,150
5,SR20,3040,150
6,HSR25,3020,90
7,SHS25,3020,96
8,SR25,3040,96
9,HSR30,3020,56


In [228]:
sql = """
SELECT manufacturing_orders.order_number, product_family, product_length, product_model, item_description, order_quantity, time_out, time_in, facility_id, machine_id
   FROM manufacturing_orders
   INNER join manufacturing_order_logs on manufacturing_orders.order_number = manufacturing_order_logs.order_number
   WHERE manufacturing_orders.order_release_code = 5
        AND facility_id = 'B0012'        
        AND time_out >= CURRENT_DATE - integer '300'
        AND product_length IS NOT NULL
        AND product_model not in ('55', '65')
        AND position('RAIL 3'in item_description) > 0        
        AND position('1778'in item_description) > 0        
"""
cut1 = call_db_json(sql)
cut_df = pd.DataFrame(cut1)
cut_df.head()

""


In [ ]:
cut_df['total'] = cut_df['product_length'] * cut_df['order_quantity'] / 1000
cut_df.groupby('item_description')['total'].sum()

In [7]:
kanban = pd.read_csv('kanan.csv', index_col=[1])

In [ ]:
path = './etc/hole_drill_machine_type.csv'
machine_type = pd.read_csv(path, index_col=[1]).reset_index()
grouped = machine_type.groupby(['Machine'])['Product'].agg(' '.join)
df = pd.DataFrame(grouped)
major = ['HSR15', 'HSR20','HSR25','HSR30','HSR35','HSR45','SR15', 'SR20','SR25','SHS15','SHS20','SHS25','SHS30','SHS35','SHS45']
print(len(major))
for m in major:
    df[m] = np.where(df['Product'].str.contains(m), 1, 0)
path= './etc/machine_occupation.csv'
# machine_occupation = pd.read_csv(path, index_col=[1]).reset_index()
machine_occupation = pd.read_csv(path)
machine_occupation